In [1]:
import os 
import pandas as pd
import numpy as np
import stanza

C:\Users\nerea\anaconda3\envs\ADA\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
text = '''In her first Reaping, 12-year-old Primrose Everdeen is chosen from
District 12. Her older sister Katniss volunteers to take her place. The nation of Panem consists of a wealthy Capitol and twelve poorer districts.
As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 
selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole
survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from
District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker's son who once gave 
Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol,
accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes
who train intensively at special academies and almost always win. During a TV interview with Caesar Flickerman, Peeta
unexpectedly reveals his love for Katniss.'''

In [3]:
nlp = stanza.Pipeline('en')

2022-11-05 19:54:55 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2022-11-05 19:55:03 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2022-11-05 19:55:03 INFO: Use device: cpu
2022-11-05 19:55:03 INFO: Loading: tokenize
2022-11-05 19:55:04 INFO: Loading: pos
2022-11-05 19:55:06 INFO: Loading: lemma
2022-11-05 19:55:06 INFO: Loading: depparse
2022-11-05 19:55:07 INFO: Loading: sentiment
2022-11-05 19:55:10 INFO: Loading: constituency
2022-11-05 19:55:13 INFO: Loading: ner
2022-11-05 19:55:17 INFO: Done loading processors!


In [9]:
doc = nlp(text)

In [10]:
for sentence in doc.sentences:
    for word in sentence.words:
        print(word.text, word.lemma, word.pos)

In in ADP
her she PRON
first first ADJ
Reaping reaping NOUN
, , PUNCT
12 12 NUM
- - PUNCT
year year NOUN
- - PUNCT
old old ADJ
Primrose Primrose PROPN
Everdeen Everdeen PROPN
is be AUX
chosen choose VERB
from from ADP
District District PROPN
12 12 NUM
. . PUNCT
Her her PRON
older old ADJ
sister sister NOUN
Katniss Katniss PROPN
volunteers volunteer NOUN
to to PART
take take VERB
her she PRON
place place NOUN
. . PUNCT
The the DET
nation nation NOUN
of of ADP
Panem Panem PROPN
consists consist VERB
of of ADP
a a DET
wealthy wealthy ADJ
Capitol Capitol PROPN
and and CCONJ
twelve twelve NUM
poorer poorer ADJ
districts district NOUN
. . PUNCT
As as ADP
punishment punishment NOUN
for for ADP
a a DET
past past ADJ
rebellion rebellion NOUN
, , PUNCT
each each DET
district district NOUN
must must AUX
provide provide VERB
a a DET
boy boy NOUN
and and CCONJ
girl girl NOUN
between between ADP
the the DET
ages age NOUN
of of ADP
12 12 NUM
and and CCONJ
18 18 NUM
selected select VERB
by by ADP
lott

In [11]:
for sentence in doc.sentences:
    #print(sentence.ents)
    print(sentence.dependencies)

[({
  "id": 4,
  "text": "Reaping",
  "lemma": "reaping",
  "upos": "NOUN",
  "xpos": "NN",
  "feats": "Number=Sing",
  "head": 14,
  "deprel": "obl",
  "start_char": 13,
  "end_char": 20
}, 'case', {
  "id": 1,
  "text": "In",
  "lemma": "in",
  "upos": "ADP",
  "xpos": "IN",
  "head": 4,
  "deprel": "case",
  "start_char": 0,
  "end_char": 2
}), ({
  "id": 4,
  "text": "Reaping",
  "lemma": "reaping",
  "upos": "NOUN",
  "xpos": "NN",
  "feats": "Number=Sing",
  "head": 14,
  "deprel": "obl",
  "start_char": 13,
  "end_char": 20
}, 'nmod:poss', {
  "id": 2,
  "text": "her",
  "lemma": "she",
  "upos": "PRON",
  "xpos": "PRP$",
  "feats": "Gender=Fem|Number=Sing|Person=3|Poss=Yes|PronType=Prs",
  "head": 4,
  "deprel": "nmod:poss",
  "start_char": 3,
  "end_char": 6
}), ({
  "id": 4,
  "text": "Reaping",
  "lemma": "reaping",
  "upos": "NOUN",
  "xpos": "NN",
  "feats": "Number=Sing",
  "head": 14,
  "deprel": "obl",
  "start_char": 13,
  "end_char": 20
}, 'amod', {
  "id": 3,
  "text

In [12]:
doc.sentences[0].print_dependencies()


print ("{:<15} | {:<10} | {:<15} ".format('Token', 'Relation', 'Head'))
print ("-" * 50)
  
# Convert sentence object to dictionary  
sent_dict = doc.sentences[0].to_dict()

# iterate to print the token, relation and head
for word in sent_dict:
  print ("{:<15} | {:<10} | {:<15} "
         .format(str(word['text']),str(word['deprel']), str(sent_dict[word['head']-1]['text'] if word['head'] > 0 else 'ROOT')))


('In', 4, 'case')
('her', 4, 'nmod:poss')
('first', 4, 'amod')
('Reaping', 14, 'obl')
(',', 4, 'punct')
('12', 8, 'nummod')
('-', 8, 'punct')
('year', 10, 'obl:npmod')
('-', 8, 'punct')
('old', 11, 'amod')
('Primrose', 14, 'nsubj:pass')
('Everdeen', 11, 'flat')
('is', 14, 'aux:pass')
('chosen', 0, 'root')
('from', 17, 'case')
('District', 17, 'compound')
('12', 14, 'obl')
('.', 14, 'punct')
Token           | Relation   | Head            
--------------------------------------------------
In              | case       | Reaping         
her             | nmod:poss  | Reaping         
first           | amod       | Reaping         
Reaping         | obl        | chosen          
,               | punct      | Reaping         
12              | nummod     | year            
-               | punct      | year            
year            | obl:npmod  | old             
-               | punct      | year            
old             | amod       | Primrose        
Primrose        | nsubj:pas